## 事件流

In [7]:
from illufly.io import EventBlock
EventBlock("Chunk", "hi")

EventBlock(block_type=<chunk>, content=<hi>)

### 实现流交换

基于大模型的AI应用中经常要求模型流式返回，我们有一种实现流输出的标准实现。

In [1]:
from illufly.types import Runnable

class MyRun(Runnable):
    def call(*args, **kwargs):
        yield "hi\n"
        yield "illufly!\n"

r = MyRun()
r()

hi
illufly!


In [4]:
from illufly.types import Runnable

class MyRun(Runnable):
    def call(*args, **kwargs):
        yield ["hi\n", "illufly!\n"]

r = MyRun()
r()

['hi\n', 'illufly!\n']

### Runnable 事件流

In [1]:
from illufly.types import Runnable, EventBlock

class MyRun(Runnable):
    def call(*args, **kwargs):
        yield EventBlock("chunk", "hi, ")
        yield EventBlock("chunk", "illufly")
        yield EventBlock("chunk", "!")

def handle(block, **kwargs):
    print(block.json)

r = MyRun(handlers=[handle])
r("hi")

{"id": "bff83a0a94694e3889927475be57c976", "content_id": "bff83a0a94694e3889927475be57c976", "block_type": "chunk", "content": "hi, ", "created_at": "2024-11-06T10:26:15.418497", "calling_info": {}, "runnable_info": {"class_name": "MyRun", "calling_id": "fd7a1423-34f9-4ebf-aa16-47c7af335821"}}
{"id": "06063b62e1724f9db72ec179c444c5b4", "content_id": "06063b62e1724f9db72ec179c444c5b4", "block_type": "chunk", "content": "illufly", "created_at": "2024-11-06T10:26:15.418726", "calling_info": {}, "runnable_info": {"class_name": "MyRun", "calling_id": "fd7a1423-34f9-4ebf-aa16-47c7af335821"}}
{"id": "f26d98f4e98d46bcb22234d51ef16521", "content_id": "f26d98f4e98d46bcb22234d51ef16521", "block_type": "chunk", "content": "!", "created_at": "2024-11-06T10:26:15.419150", "calling_info": {}, "runnable_info": {"class_name": "MyRun", "calling_id": "fd7a1423-34f9-4ebf-aa16-47c7af335821"}}


### ChatAgent 事件流

In [7]:
from illufly.types import Runnable, EventBlock
from illufly.chat import FakeLLM

def handle(block, **kwargs):
    print(block.json)

r = FakeLLM(handlers=[handle])
r("hi")

{"id": "716128e9cbc342b9a9a03093f9418178", "content_id": "716128e9cbc342b9a9a03093f9418178", "block_type": "user", "content": "hi", "created_at": "2024-11-06T10:32:20.400669", "calling_info": {}, "runnable_info": {"class_name": "FakeLLM", "calling_id": "12e70938-a7c6-4437-b579-698374c1121c", "agent_name": "FakeLLM.4817163344", "agent_description": "我还没有工具描述", "thread_id": null, "model_name": null, "base_url": null, "api_key": null}}
{"id": "0b473d7fa5da431ab6f47c0b9a972bce", "content_id": "0b473d7fa5da431ab6f47c0b9a972bce", "block_type": "info", "content": "记住 10 轮对话", "created_at": "2024-11-06T10:32:20.400941", "calling_info": {}, "runnable_info": {"class_name": "FakeLLM", "calling_id": "12e70938-a7c6-4437-b579-698374c1121c", "agent_name": "FakeLLM.4817163344", "agent_description": "我还没有工具描述", "thread_id": null, "model_name": null, "base_url": null, "api_key": null}}
{"id": "21eadae276324ea597a55f92338e8f07", "content_id": "21eadae276324ea597a55f92338e8f07", "block_type": "info", "con

'Reply >> hi'

### 提取整洁的 ChatAgent 事件流

In [1]:
from illufly.types import Runnable, EventBlock
from illufly.chat import FakeLLM

def handle(block, **kwargs):
    r = block.runnable_info
    if block.block_type in ["user", "chunk"]:
        print(f'@{r["agent_name"]} [{r["thread_id"]}/{block.content_id}] {block.text}')

r = FakeLLM(handlers=[handle])
r("hi")

@FakeLLM.4426165232 [None/7a979bc49c4611efa9cf367dda7bcd4a] hi
@FakeLLM.4426165232 [901199-3037-0000/7a97ab009c4611efa9cf367dda7bcd4a] R
@FakeLLM.4426165232 [901199-3037-0000/7a97ab009c4611efa9cf367dda7bcd4a] e
@FakeLLM.4426165232 [901199-3037-0000/7a97ab009c4611efa9cf367dda7bcd4a] p
@FakeLLM.4426165232 [901199-3037-0000/7a97ab009c4611efa9cf367dda7bcd4a] l
@FakeLLM.4426165232 [901199-3037-0000/7a97ab009c4611efa9cf367dda7bcd4a] y
@FakeLLM.4426165232 [901199-3037-0000/7a97ab009c4611efa9cf367dda7bcd4a]  
@FakeLLM.4426165232 [901199-3037-0000/7a97ab009c4611efa9cf367dda7bcd4a] >
@FakeLLM.4426165232 [901199-3037-0000/7a97ab009c4611efa9cf367dda7bcd4a] >
@FakeLLM.4426165232 [901199-3037-0000/7a97ab009c4611efa9cf367dda7bcd4a]  
@FakeLLM.4426165232 [901199-3037-0000/7a97ab009c4611efa9cf367dda7bcd4a] h
@FakeLLM.4426165232 [901199-3037-0000/7a97ab009c4611efa9cf367dda7bcd4a] i


'Reply >> hi'

In [2]:
r("你好")

@FakeLLM.4573043856 [863301-9636-0000/2b63301b2437430185c1472799d45775] 你好
@FakeLLM.4573043856 [863301-9636-0000/19eec31a231a4f90b9142dc74a949402] R
@FakeLLM.4573043856 [863301-9636-0000/19eec31a231a4f90b9142dc74a949402] e
@FakeLLM.4573043856 [863301-9636-0000/19eec31a231a4f90b9142dc74a949402] p
@FakeLLM.4573043856 [863301-9636-0000/19eec31a231a4f90b9142dc74a949402] l
@FakeLLM.4573043856 [863301-9636-0000/19eec31a231a4f90b9142dc74a949402] y
@FakeLLM.4573043856 [863301-9636-0000/19eec31a231a4f90b9142dc74a949402]  
@FakeLLM.4573043856 [863301-9636-0000/19eec31a231a4f90b9142dc74a949402] >
@FakeLLM.4573043856 [863301-9636-0000/19eec31a231a4f90b9142dc74a949402] >
@FakeLLM.4573043856 [863301-9636-0000/19eec31a231a4f90b9142dc74a949402]  
@FakeLLM.4573043856 [863301-9636-0000/19eec31a231a4f90b9142dc74a949402] 你
@FakeLLM.4573043856 [863301-9636-0000/19eec31a231a4f90b9142dc74a949402] 好


'Reply >> 你好'

In [1]:
from illufly.chat import ChatQwen
from illufly.io import EventBlock, create_collector
from typing import Union, List, Set

def get_current_weather(location: Set[str]):
    """获取城市的天气情况"""
    return f"{location}今天是晴天。 "


store = {}

def handle(block, **kwargs):
    r = block.runnable_info
    if block.block_type in ["user", "chunk", "text", "tool_resp_chunk", "tool_resp_text"]:
        print(f'@{r["agent_name"]} <{block.block_type}> [{r["thread_id"]}/{block.content_id}] {block.text}')
    else:
        print(f'@{r["agent_name"]} <{block.block_type}>')

qwen = ChatQwen(tools=[get_current_weather], handlers=[handle, create_collector(store)], name="qwen")

qwen("我在广州，今天可以到顶楼晒衣服吗？", thread_id=qwen.create_new_thread())
store

@qwen <user> [901342-5246-0000/cfdd7d749c4611ef844a367dda7bcd4a] 我在广州，今天可以到顶楼晒衣服吗？
@qwen <info>
@qwen <tools_call_chunk>
@qwen <tools_call_chunk>
@qwen <tools_call_chunk>
@qwen <tools_call_chunk>
@qwen <tools_call_chunk>
@qwen <new_line>
@qwen <usage>
@qwen <new_line>
@qwen <final_tools_call>
@qwen <agent>
@qwen <tool_resp_text> [901342-5246-0000/cfdd8f449c4611ef844a367dda7bcd4a] ['广州']今天是晴天。 
@qwen <new_line>
@qwen <final_tool_resp>
@qwen <chunk> [901342-5246-0000/d0b87d029c4611ef844a367dda7bcd4a] 今天
@qwen <chunk> [901342-5246-0000/d0b87d029c4611ef844a367dda7bcd4a] 广州
@qwen <chunk> [901342-5246-0000/d0b87d029c4611ef844a367dda7bcd4a] 是晴天，
@qwen <chunk> [901342-5246-0000/d0b87d029c4611ef844a367dda7bcd4a] 所以你可以到顶
@qwen <chunk> [901342-5246-0000/d0b87d029c4611ef844a367dda7bcd4a] 楼晒衣服。
@qwen <chunk> [901342-5246-0000/d0b87d029c4611ef844a367dda7bcd4a] 
@qwen <chunk> [901342-5246-0000/d0b87d029c4611ef844a367dda7bcd4a] 
@qwen <new_line>
@qwen <usage>
@qwen <final_text>


{'901342-5246-0000': {'cfdd78b0-9c46-11ef-844a-367dda7bcd4a': {'input': '我在广州，今天可以到顶楼晒衣服吗？',
   'output': '今天广州是晴天，所以你可以到顶楼晒衣服。',
   'segments': {'cfdd8f449c4611ef844a367dda7bcd4a': "['广州']今天是晴天。 ",
    'd0b87d029c4611ef844a367dda7bcd4a': '今天广州是晴天，所以你可以到顶楼晒衣服。'},
   'other_events': []}}}

## 嵌套 BaseAgent

In [5]:
from illufly.types import BaseAgent, PromptTemplate
from illufly.chat import ChatQwen
from illufly.flow import ReAct
from illufly.io import create_collector, log

def get_city(location: str):
    """由任意地名或地址描述查询出所在的城市"""
    return "广州"

def get_weather(city: str):
    """我可以查询城市的天气情况。city必须是城市名称。"""
    return f'{city}天气不错，一直是大晴天'

def booking(request: str):
    """我可以根据你的需求预订球场"""
    return '我已经帮你预订好了，祝你玩得愉快'

store = {}
flow = ReAct(
    ChatQwen(name="react-planner", tools=[get_city, get_weather, booking]),
    handlers=[log, create_collector(store)]
)

flow("我在鸿蒙公司，帮我查询一下天气情况", verbose=False)
store

[AGENT] >>> Node 1: planner
**思考** 

首先，我需要确定鸿蒙公司所在的地理位置，以便查询该地区的天气情况。因此，我的第一步是使用 `get_city` 工具来获取鸿蒙公司所在的城市。

任务中涉及的关键概念包括：
- 鸿蒙公司
 - 所在城市

**行动**

Step1: 查询鸿蒙公司所在的城市. #E1 = get_city[{"location": "鸿蒙公司"}]
[AGENT] get_city
广州
[AGENT] >>> Node 2: observer

**观察**
上面的行动结果为:
广州
[AGENT] >>> Node 1: planner
**思考** 
既然已经知道了鸿蒙公司位于广州，那么接下来的任务就是查询广州的天气情况。

任务中涉及的关键概念包括：
- 鸿蒙公司 - 广州
 - 所在城市 - 广州

**行动** 
Step1: 查询广州的天气情况. #E1 = get_weather[{"city": "广州"}]
[AGENT] get_weather
广州天气不错，一直是大晴天
[AGENT] >>> Node 2: observer

**观察**
上面的行动结果为:
广州天气不错，一直是大晴天
[AGENT] >>> Node 1: planner
**思考** 
目前，我们已经知道鸿蒙公司位于广州，并且了解到广州的天气情况。因此，我们可以直接输出最终答案。

任务中涉及的关键概念包括：
- 鸿蒙公司 - 广州
 - 所在城市 - 广州

**最终答案**
鸿蒙公司位于广州，广州的天气不错，一直是大晴天。
[AGENT] >>> Node 2: observer


{'__default__': {'5dab58c4-9c47-11ef-844a-367dda7bcd4a': {'input': '我在鸿蒙公司，帮我查询一下天气情况',
   'output': '我在鸿蒙公司，帮我查询一下天气情况',
   'segments': {'5dab7c289c4711ef844a367dda7bcd4a': '**思考** \n\n首先，我需要确定鸿蒙公司所在的地理位置，以便查询该地区的天气情况。因此，我的第一步是使用 `get_city` 工具来获取鸿蒙公司所在的城市。\n\n任务中涉及的关键概念包括：\n- 鸿蒙公司\n - 所在城市\n\n**行动**\n\nStep1: 查询鸿蒙公司所在的城市. #E1 = get_city[{"location": "鸿蒙公司"}]广州',
    '60a4c7549c4711ef844a367dda7bcd4a': '\n**观察**\n上面的行动结果为:\n广州\n',
    '60a504d09c4711ef844a367dda7bcd4a': '**思考** \n既然已经知道了鸿蒙公司位于广州，那么接下来的任务就是查询广州的天气情况。\n\n任务中涉及的关键概念包括：\n- 鸿蒙公司 - 广州\n - 所在城市 - 广州\n\n**行动** \nStep1: 查询广州的天气情况. #E1 = get_weather[{"city": "广州"}]广州天气不错，一直是大晴天',
    '645ec2509c4711ef844a367dda7bcd4a': '\n**观察**\n上面的行动结果为:\n广州天气不错，一直是大晴天\n',
    '645ef7529c4711ef844a367dda7bcd4a': '**思考** \n目前，我们已经知道鸿蒙公司位于广州，并且了解到广州的天气情况。因此，我们可以直接输出最终答案。\n\n任务中涉及的关键概念包括：\n- 鸿蒙公司 - 广州\n - 所在城市 - 广州\n\n**最终答案**\n鸿蒙公司位于广州，广州的天气不错，一直是大晴天。'},
   'other_events': []}}}